<a href="https://colab.research.google.com/github/youcef-4/VisDrone/blob/main/tph_yolov5_on_VisDrone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/youcef-4/VisDrone.git

Cloning into 'VisDrone'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (268/268), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 268 (delta 126), reused 242 (delta 112), pack-reused 0
Receiving objects: 100% (268/268), 9.28 MiB | 39.77 MiB/s, done.
Resolving deltas: 100% (126/126), done.


In [2]:
!cd tph-yolov5

/bin/bash: line 0: cd: tph-yolov5: No such file or directory


In [3]:
!pip install -r /content/VisDrone/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=f201d6f97b4385dc7cdf865aaaeee43d7f7e8e2d80d9c16f7ce553a076a7c430
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!python /content/VisDrone/VisDrone2YOLO_lable.py

548


In [6]:
#!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

In [7]:
#!sudo update-alternatives --config python3

In [8]:
#! python --version

In [9]:
#!pip3 install clean-fid numba numpy torch==2.0.0+cu118 torchvision --force-reinstall --extra-index-url https://download.pytorch.org/whl/cu118

In [10]:
#print(torch.__version__)

In [11]:
#!pip install opencv-python>=4.1.2

In [22]:
#!/bin/bash

# Define the file path
FILE_PATH="/usr/local/lib/python3.10/dist-packages/torch/nn/modules/upsampling.py"

# Make a backup of the original file
!cp $FILE_PATH "${FILE_PATH}.bak"

# Use sed to comment out the line
!sed -i 's/\(recompute_scale_factor=self.recompute_scale_factor\)/#\1\n)/g' $FILE_PATH


cp: cannot create regular file '$/usr/local/lib/python3.10/dist-packages/torch/nn/modules/upsampling.py.bak': No such file or directory


In [25]:
!python /content/VisDrone/val.py --weights /content/drive/MyDrive/VisDrone/yolov5l-xs-2.pt --img 1996 --data /content/VisDrone/data/VisDrone.yaml --augment --save-txt  --save-conf --task val --batch-size 8 --verbose --name v5l-xs

val: data=/content/VisDrone/data/VisDrone.yaml, weights=['/content/drive/MyDrive/VisDrone/yolov5l-xs-2.pt'], batch_size=8, imgsz=1996, conf_thres=0.001, iou_thres=0.6, task=val, device=, single_cls=False, augment=True, verbose=True, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=VisDrone/runs/val, name=v5l-xs, exist_ok=False, half=False
YOLOv5 🚀 2757b98 torch 2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model Summary: 589 layers, 68030516 parameters, 0 gradients, 151.8 GFLOPs
val: Scanning '../datasets/VisDrone/VisDrone2019-DET-val/labels' images and labels...548 found, 0 missing, 0 empty, 0 corrupted: 100% 548/548 [00:00<00:00, 1640.82it/s]
val: New cache created: ../datasets/VisDrone/VisDrone2019-DET-val/labels.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 69/69 [17:30<00:00, 15.22s/it]
                 all        548      38759       0.66      0.603      0.632       0.41
          pedestri

In [36]:
!pip install -U scikit-learn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
from sklearn.metrics import confusion_matrix

def generate_confusion_matrix(preds_path, labels_path, class_names):
    """
    Generate confusion matrix for the VisDrone dataset.

    :param preds_path: Path to directory containing predicted bounding boxes in YOLO format
    :param labels_path: Path to directory containing ground truth bounding boxes in YOLO format
    :param class_names: List of class names
    """
    num_classes = len(class_names)
    y_true = []
    y_pred = []

    # Iterate over all files in the labels directory
    for label_file in Path(labels_path).glob('*.txt'):
        # Load ground truth labels
        gt_labels = np.atleast_2d(np.genfromtxt(label_file, delimiter=','))

        # Load predicted labels
        pred_file = Path(preds_path) / label_file.name
        if pred_file.exists():
            pred_labels = np.atleast_2d(np.genfromtxt(pred_file, delimiter=','))
        else:
            continue

        # Add ground truth and predicted labels to the lists
        y_true.extend(gt_labels[:, 0].tolist())
        y_pred.extend(pred_labels[:, 0].tolist())

    # Compute multi-class confusion matrix
    conf_mat = confusion_matrix(y_true, y_pred, labels=range(num_classes))

    # Print confusion matrix
    print(f"Confusion matrix:")
    print(conf_mat)

# Example usage
preds_path = '/content/VisDrone/runs/val/v5l-xs2/labels'
labels_path = '/datasets/VisDrone/VisDrone2019-DET-val/labels'
class_names = ['pedestrian', 'people', 'bicycle', 'car', 'van', 'truck', 'tricycle', 'awning-tricycle', 'bus', 'motor']
generate_confusion_matrix(preds_path, labels_path, class_names)


ValueError: ignored